In [4]:
# NOTE: The way the functions are arranged 
# is a bit repetitive, you can group method in modules
# then you can from npmine.util import something
from npmine.retrieve_doi import retrieve_doi

import json

In [2]:
# Retrieve all DOis from this journal
# NOTE: The function should contain the parameter referring to the journal
doi = retrieve_doi()

In [3]:
doi

['https://pubs.acs.org/doi/pdf/10.1021/np50001a700',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a001',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a002',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a003',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a004',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a005',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a006',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a007',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a008',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a009',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a010',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a700',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a001',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a002',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a003',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a004',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a005',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002a006',
 'https://pubs.acs.org/doi/pdf/10.1021/np50002

In [14]:
# write doi file
with open('../data/doi_dnp.json', 'w+') as outfile:
    json.dump(doi, outfile, indent=4)

# compress the json file
!zip ../data/doi_dnp.zip ../data/doi_dnp.json

  adding: ../data/doi_dnp.json (deflated 93%)


In [11]:
import os
os.getcwd()

'/home/rsilva/Documents/npmine_library/notebooks'

In [16]:
# uncompress the json file
!unzip ../data/doi_dnp.zip -d ../data/

# Load the file, so you don't need to generate it
# every time
with open('../data/doi_dnp.json', 'r') as inputfile:
    doi = json.load(inputfile)

Archive:  ../data/doi_dnp.zip
  inflating: ../data/data/doi_dnp.json  


In [17]:
len(doi)

14481